In [7]:
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 13 10:02:35 2023

@author: admin
"""

import psycopg2
import time
import pandas as pd
from io import StringIO
import re
import os
from os import walk
from datetime import date

st=time.time()

date1 = input('Enter a date in YYYY-MM-DD format ')                                     ## TO BE CHANGED DAILY AS PER UPDATION DATE
year , month , day = map(int,date1.split('-'))
day , month = f"{day:02d}",f"{month:02d}"

# date1 = date(2023,5,4)
# day = date1.strftime('%d')
# nummonth=date1.strftime("%m")
# year=date1.strftime('%Y')
datestr=str(year)+"-"+str(month)+"-"+str(day)
print(datestr)
choice = int(input("1 for BANKNIFTY\n2 for NIFTY\n3 for FINNIFTY\n4 to UPDATE ALL\n5 to EXIT\n"))

def banknifty_monthly():
    conn = psycopg2.connect(database="BankNiftydb",
    						user='postgres', password='swancap123',
    						host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com', port='5432'
    )
    conn.autocommit = True
    cursor = conn.cursor()
    print("APPENDING BANKNIFTY MONTHLY CONTRACTS")
    for i in range(1,4):
        if(i==1):
            numberstring="I"
            schema="BankNiftyMonthlyI"
            infotable="BANKNIFTYMONTHLY-Iinfo"
        if(i==2):
            numberstring="II"
            schema="BankNiftyMonthlyII"
            infotable="BANKNIFTYMONTHLY-IIinfo"
        if(i==3):
            numberstring="III"
            schema="BankNiftyMonthlyIII"
            infotable="BANKNIFTYMONTHLY-IIIinfo"
            
        bndf=pd.read_csv("C:\\Users\\admin\\Desktop\\Pyspark\\BankNifty\Monthly\Banknifty-"+numberstring+".csv")
        df=bndf.groupby(['Ticker'])
        for name,group in df:
            print(name)
            sql2='''CREATE TABLE IF NOT EXISTS "'''+schema+'''"."'''+name+'''"(Ticker varchar(50) NOT NULL,Date Date NOT NULL,Time time NOT NULL,Open float NOT NULL,High float NOT NULL,Low float NOT NULL,Close float NOT NULL,Volume float NOT NULL,"Open Int" float NOT NULL);'''
            cursor.execute(sql2)
            conn.commit()
            noofrows=group.shape[0]
#             print(noofrows)
            buffer = StringIO()
            group.to_csv(buffer, index = False)
            buffer.seek(0)
            sql = "COPY %s FROM STDIN WITH CSV HEADER DELIMITER AS ','"
            table='"'+schema+'"."'+name+'"'
            with conn.cursor() as cur:
                #cur.execute("truncate " + table + ";")
                cur.copy_expert(sql=sql % table, file=buffer)
                conn.commit()
            cursor = conn.cursor()
            s='''Select 1 from "'''+schema+'''"."'''+infotable+'''" where ticker=\''''+name+'''\';'''
            cursor.execute(s)
            k=cursor.fetchall()
#             print(k)
            if(k==[]):
                num=re.findall(r"[-+]?(?:\d*\.*\d+)", name)
                strike=num[0]
                typecepe=name[-2:]
                sql3 ='''INSERT INTO "'''+schema+'''"."'''+infotable+'''"(Ticker,Strike,Type)VALUES (%s,%s,%s)'''
                record_to_insert = (name,strike,typecepe)
                cursor.execute(sql3, record_to_insert)
                conn.commit()
            sql4='''Select count(*) from "'''+schema+'''"."'''+name+'''" where date=\''''+datestr+'''\';'''
            cursor.execute(sql4)
            r=cursor.fetchall()
#             print(r[0][0])
            if(r[0][0]!=noofrows):
                print("numberofrows not matching")
                break
    conn.close()

def banknifty_weekly():
    conn = psycopg2.connect(database="BankNiftydb",
    						user='postgres', password='swancap123',
    						host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com', port='5432'
    )
    conn.autocommit = True
    cursor = conn.cursor()
    print("APPENDING BANKNIFTY WEEKLY CONTRACTS")
    for i in range(1,15):
        if(i==1):
            numberstring="I"
            schema="BankNiftyWeeklyI"
            infotable="BANKNIFTYWEEKLY-Iinfo"
        if(i==2):
            numberstring="II"
            schema="BankNiftyWeeklyII"
            infotable="BANKNIFTYWEEKLY-IIinfo"
        if(i==3):
            numberstring="III"
            schema="BankNiftyWeeklyIII"
            infotable="BANKNIFTYWEEKLY-IIIinfo"
        if(i==4):
            numberstring="IV"
            schema="BankNiftyWeeklyIV"
            infotable="BANKNIFTYWEEKLY-IVinfo"
        if(i==5):
            numberstring="V"
            schema="BankNiftyWeeklyV"
            infotable="BANKNIFTYWEEKLY-Vinfo"
        if(i==6):
            numberstring="VI"
            schema="BankNiftyWeeklyVI"
            infotable="BANKNIFTYWEEKLY-VIinfo"
        if(i==7):
            numberstring="VII"
            schema="BankNiftyWeeklyVII"
            infotable="BANKNIFTYWEEKLY-VIIinfo"
        if(i==8):
            numberstring="VIII"
            schema="BankNiftyWeeklyVIII"
            infotable="BANKNIFTYWEEKLY-VIIIinfo"
        if(i==9):
            numberstring="IX"
            schema="BankNiftyWeeklyIX"
            infotable="BANKNIFTYWEEKLY-IXinfo"
        if(i==10):
            numberstring="X"
            schema="BankNiftyWeeklyX"
            infotable="BANKNIFTYWEEKLY-Xinfo"
        if(i==11):
            numberstring="XI"
            schema="BankNiftyWeeklyXI"
            infotable="BANKNIFTYWEEKLY-XIinfo"
        if(i==12):
            numberstring="XII"
            schema="BankNiftyWeeklyXII"
            infotable="BANKNIFTYWEEKLY-XIIinfo"
        if(i==13):
            numberstring="XIII"
            schema="BankNiftyWeeklyXIII"
            infotable="BANKNIFTYWEEKLY-XIIIinfo"
        if(i==14):
            numberstring="XIV"
            schema="BankNiftyWeeklyXIV"
            infotable="BANKNIFTYWEEKLY-XIVinfo"
    
        if os.path.exists(r"C:\Users\admin\Desktop\Pyspark\BankNifty\Weekly\Banknifty-"+numberstring+".csv"):
            bndf=pd.read_csv("C:\\Users\\admin\\Desktop\\Pyspark\BankNifty\Weekly\\Banknifty-"+numberstring+".csv")
            df=bndf.groupby(['Ticker'])
            for name,group in df:
                print(name)
                sql2='''CREATE TABLE IF NOT EXISTS "'''+schema+'''"."'''+name+'''"(Ticker varchar(50) NOT NULL,Date Date NOT NULL,Time time NOT NULL,Open float NOT NULL,High float NOT NULL,Low float NOT NULL,Close float NOT NULL,Volume float NOT NULL,"Open Int" float NOT NULL);'''
                cursor.execute(sql2)
                conn.commit()
                noofrows=group.shape[0]
#                 print(noofrows)
                buffer = StringIO()
                group.to_csv(buffer, index = False)
                buffer.seek(0)
                sql = "COPY %s FROM STDIN WITH CSV HEADER DELIMITER AS ','"
                table='"'+schema+'"."'+name+'"'
                with conn.cursor() as cur:
                    #cur.execute("truncate " + table + ";")
                    cur.copy_expert(sql=sql % table, file=buffer)
                    conn.commit()
                cursor = conn.cursor()
                s='''Select 1 from "'''+schema+'''"."'''+infotable+'''" where ticker=\''''+name+'''\';'''
                cursor.execute(s)
                k=cursor.fetchall()
#                 print(k)
                if(k==[]):
                    num=re.findall(r"[-+]?(?:\d*\.*\d+)", name)
                    strike=num[0]
                    typecepe=name[-2:]
                    sql3 ='''INSERT INTO "'''+schema+'''"."'''+infotable+'''"(Ticker,Strike,Type)VALUES (%s,%s,%s)'''
                    record_to_insert = (name,strike,typecepe)
                    cursor.execute(sql3, record_to_insert)
                    conn.commit()
                sql4='''Select count(*) from "'''+schema+'''"."'''+name+'''" where date=\''''+datestr+'''\';'''
                cursor.execute(sql4)
                r=cursor.fetchall()
#                 print(r[0][0])
                if(r[0][0]!=noofrows):
                    print("numberofrows not matching")
                    break
    conn.close()

def banknifty_quarterly():
    conn = psycopg2.connect(database="BankNiftydb",
    						user='postgres', password='swancap123',
    						host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com', port='5432'
    )
    conn.autocommit = True
    cursor = conn.cursor()
    print("APPENDING BANKNIFTY QUARTERLY CONTRACTS")
    for i in range(1,9):
        if(i==1):
            numberstring="I"
            schema="BankNiftyQuarterlyI"
            infotable="BANKNIFTYQUARTERLY-Iinfo"
        if(i==2):
            numberstring="II"
            schema="BankNiftyQuarterlyII"
            infotable="BANKNIFTYQUARTERLY-IIinfo"
        if(i==3):
            numberstring="III"
            schema="BankNiftyQuarterlyIII"
            infotable="BANKNIFTYQUARTERLY-IIIinfo"
        if(i==4):
            numberstring="IV"
            schema="BankNiftyQuarterlyIV"
            infotable="BANKNIFTYQUARTERLY-IVinfo"
        if(i==5):
            numberstring="V"
            schema="BankNiftyQuarterlyV"
            infotable="BANKNIFTYQUARTERLY-Vinfo"
        if(i==6):
            numberstring="VI"
            schema="BankNiftyQuarterlyVI"
            infotable="BANKNIFTYQUARTERLY-VIinfo"
        if(i==7):
            numberstring="VII"
            schema="BankNiftyQuarterlyVII"
            infotable="BANKNIFTYQUARTERLY-VIIinfo"
        if(i==8):
            numberstring="VIII"
            schema="BankNiftyQuarterlyVIII"
            infotable="BANKNIFTYQUARTERLY-VIIIinfo"
   
        if os.path.exists(r"C:\Users\admin\Desktop\Pyspark\BankNifty\Quarterly\Banknifty-"+numberstring+".csv"):
            bndf=pd.read_csv("C:\\Users\\admin\\Desktop\\Pyspark\BankNifty\Quarterly\\Banknifty-"+numberstring+".csv")
            df=bndf.groupby(['Ticker'])
            for name,group in df:
                print(name)
                sql2='''CREATE TABLE IF NOT EXISTS "'''+schema+'''"."'''+name+'''"(Ticker varchar(50) NOT NULL,Date Date NOT NULL,Time time NOT NULL,Open float NOT NULL,High float NOT NULL,Low float NOT NULL,Close float NOT NULL,Volume float NOT NULL,"Open Int" float NOT NULL);'''
                cursor.execute(sql2)
                conn.commit()
                noofrows=group.shape[0]
#                 print(noofrows)
                buffer = StringIO()
                group.to_csv(buffer, index = False)
                buffer.seek(0)
                sql = "COPY %s FROM STDIN WITH CSV HEADER DELIMITER AS ','"
                table='"'+schema+'"."'+name+'"'
                with conn.cursor() as cur:
                    #cur.execute("truncate " + table + ";")
                    cur.copy_expert(sql=sql % table, file=buffer)
                    conn.commit()
                cursor = conn.cursor()
                s='''Select 1 from "'''+schema+'''"."'''+infotable+'''" where ticker=\''''+name+'''\';'''
                cursor.execute(s)
                k=cursor.fetchall()
#                 print(k)
                if(k==[]):
                    num=re.findall(r"[-+]?(?:\d*\.*\d+)", name)
                    strike=num[0]
                    typecepe=name[-2:]
                    sql3 ='''INSERT INTO "'''+schema+'''"."'''+infotable+'''"(Ticker,Strike,Type)VALUES (%s,%s,%s)'''
                    record_to_insert = (name,strike,typecepe)
                    cursor.execute(sql3, record_to_insert)
                    conn.commit()
                sql4='''Select count(*) from "'''+schema+'''"."'''+name+'''" where date=\''''+datestr+'''\';'''
                cursor.execute(sql4)
                r=cursor.fetchall()
#                 print(r[0][0])
                if(r[0][0]!=noofrows):
                    print("numberofrows not matching")
                    break
    conn.close()
    
def banknifty_halfyearly():
    conn = psycopg2.connect(database="BankNiftydb",
    						user='postgres', password='swancap123',
    						host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com', port='5432'
    )
    conn.autocommit = True
    cursor = conn.cursor()
    print("APPENDING BANKNIFTY HALFYEARLY CONTRACTS")
    for i in range(1,5):
        if(i==1):
            numberstring="I"
            schema="BankNiftyHalfYearlyI"
            infotable="BANKNIFTYHALFYEARLY-IInfo"
        if(i==2):
            numberstring="II"
            schema="BankNiftyHalfYearlyII"
            infotable="BANKNIFTYHALFYEARLY-IIInfo"
        if(i==3):
            numberstring="III"
            schema="BankNiftyHalfYearlyIII"
            infotable="BANKNIFTYHALFYEARLY-IIIInfo"
        if(i==4):
            numberstring="IV"
            schema="BankNiftyHalfYearlyIV"
            infotable="BANKNIFTYHALFYEARLY-IVInfo"
    
        if os.path.exists(r"C:\Users\admin\Desktop\Pyspark\BankNifty\Half_Yearly\Banknifty-"+numberstring+".csv"):
            bndf=pd.read_csv("C:\\Users\\admin\\Desktop\\Pyspark\BankNifty\Half_Yearly\\Banknifty-"+numberstring+".csv")
            df=bndf.groupby(['Ticker'])
            for name,group in df:
                print(name)
                sql2='''CREATE TABLE IF NOT EXISTS "'''+schema+'''"."'''+name+'''"(Ticker varchar(50) NOT NULL,Date Date NOT NULL,Time time NOT NULL,Open float NOT NULL,High float NOT NULL,Low float NOT NULL,Close float NOT NULL,Volume float NOT NULL,"Open Int" float NOT NULL);'''
                cursor.execute(sql2)
                conn.commit()
                noofrows=group.shape[0]
#                 print(noofrows)
                buffer = StringIO()
                group.to_csv(buffer, index = False)
                buffer.seek(0)
                sql = "COPY %s FROM STDIN WITH CSV HEADER DELIMITER AS ','"
                table='"'+schema+'"."'+name+'"'
                with conn.cursor() as cur:
                    #cur.execute("truncate " + table + ";")
                    cur.copy_expert(sql=sql % table, file=buffer)
                    conn.commit()
                cursor = conn.cursor()
                s='''Select 1 from "'''+schema+'''"."'''+infotable+'''" where ticker=\''''+name+'''\';'''
                cursor.execute(s)
                k=cursor.fetchall()
#                 print(k)
                if(k==[]):
                    num=re.findall(r"[-+]?(?:\d*\.*\d+)", name)
                    strike=num[0]
                    typecepe=name[-2:]
                    sql3 ='''INSERT INTO "'''+schema+'''"."'''+infotable+'''"(Ticker,Strike,Type)VALUES (%s,%s,%s)'''
                    record_to_insert = (name,strike,typecepe)
                    cursor.execute(sql3, record_to_insert)
                    conn.commit()
                sql4='''Select count(*) from "'''+schema+'''"."'''+name+'''" where date=\''''+datestr+'''\';'''
                cursor.execute(sql4)
                r=cursor.fetchall()
#                 print(r[0][0])
                if(r[0][0]!=noofrows):
                    print("numberofrows not matching")
                    break
    conn.close()

def banknifty_yearly():
    conn = psycopg2.connect(database="BankNiftydb",
    						user='postgres', password='swancap123',
    						host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com', port='5432'
    )
    conn.autocommit = True
    cursor = conn.cursor()
    print("APPENDING BANKNIFTY YEARLY CONTRACTS")
    for i in range(1,3):
        if(i==1):
            numberstring="I"
            schema="BankNiftyYearlyI"
            infotable="BANKNIFTYYEARLY-IInfo"
        if(i==2):
            numberstring="II"
            schema="BankNiftyYearlyII"
            infotable="BANKNIFTYYEARLY-IIInfo"
            
        if os.path.exists(r"C:\Users\admin\Desktop\Pyspark\BankNifty\Yearly\Banknifty-"+numberstring+".csv"):
            bndf=pd.read_csv("C:\\Users\\admin\\Desktop\\Pyspark\BankNifty\Yearly\\Banknifty-"+numberstring+".csv")
            df=bndf.groupby(['Ticker'])
            for name,group in df:
                print(name)
                sql2='''CREATE TABLE IF NOT EXISTS "'''+schema+'''"."'''+name+'''"(Ticker varchar(50) NOT NULL,Date Date NOT NULL,Time time NOT NULL,Open float NOT NULL,High float NOT NULL,Low float NOT NULL,Close float NOT NULL,Volume float NOT NULL,"Open Int" float NOT NULL);'''
                cursor.execute(sql2)
                conn.commit()
                noofrows=group.shape[0]
#                 print(noofrows)
                buffer = StringIO()
                group.to_csv(buffer, index = False)
                buffer.seek(0)
                sql = "COPY %s FROM STDIN WITH CSV HEADER DELIMITER AS ','"
                table='"'+schema+'"."'+name+'"'
                with conn.cursor() as cur:
                    #cur.execute("truncate " + table + ";")
                    cur.copy_expert(sql=sql % table, file=buffer)
                    conn.commit()
                cursor = conn.cursor()
                s='''Select 1 from "'''+schema+'''"."'''+infotable+'''" where ticker=\''''+name+'''\';'''
                cursor.execute(s)
                k=cursor.fetchall()
#                 print(k)
                if(k==[]):
                    num=re.findall(r"[-+]?(?:\d*\.*\d+)", name)
                    strike=num[0]
                    typecepe=name[-2:]
                    sql3 ='''INSERT INTO "'''+schema+'''"."'''+infotable+'''"(Ticker,Strike,Type)VALUES (%s,%s,%s)'''
                    record_to_insert = (name,strike,typecepe)
                    cursor.execute(sql3, record_to_insert)
                    conn.commit()
                sql4='''Select count(*) from "'''+schema+'''"."'''+name+'''" where date=\''''+datestr+'''\';'''
                cursor.execute(sql4)
                r=cursor.fetchall()
#                 print(r[0][0])
                if(r[0][0]!=noofrows):
                    print("numberofrows not matching")
                    break
    conn.close()

def nifty_monthly():
    conn = psycopg2.connect(database="Niftydb",
                            user='postgres', password='swancap123',
                            host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com', port='5432'
    )
    conn.autocommit = True
    cursor = conn.cursor()
    print("APPENDING NIFTY MONTHLY CONTRACTS")
    for i in range(1,4):
        if(i==1):
            numberstring="I"
            schema="NiftyMonthlyI"
            infotable="NIFTYMONTHLY-IInfo"
        if(i==2):
            numberstring="II"
            schema="NiftyMonthlyII"
            infotable="NIFTYMONTHLY-IIInfo"
        if(i==3):
            numberstring="III"
            schema="NiftyMonthlyIII"
            infotable="NIFTYMONTHLY-IIIInfo"
        bndf=pd.read_csv("C:\\Users\\admin\\Desktop\\Pyspark\\Nifty\\Monthly\\NIFTY-"+numberstring+".csv")
        df=bndf.groupby(['Ticker'])
        for name,group in df:
            print(name)
            sql2='''CREATE TABLE IF NOT EXISTS "'''+schema+'''"."'''+name+'''"(Ticker varchar(50) NOT NULL,Date Date NOT NULL,Time time NOT NULL,Open float NOT NULL,High float NOT NULL,Low float NOT NULL,Close float NOT NULL,Volume float NOT NULL,"Open Int" float NOT NULL);'''
            cursor.execute(sql2)
            conn.commit()
            noofrows=group.shape[0]
#             print(noofrows)
            buffer = StringIO()
            group.to_csv(buffer, index = False)
            buffer.seek(0)
            sql = "COPY %s FROM STDIN WITH CSV HEADER DELIMITER AS ','"
            table='"'+schema+'"."'+name+'"'
            with conn.cursor() as cur:
                #cur.execute("truncate " + table + ";")
                cur.copy_expert(sql=sql % table, file=buffer)
                conn.commit()
            cursor = conn.cursor()
            s='''Select 1 from "'''+schema+'''"."'''+infotable+'''" where ticker=\''''+name+'''\';'''
            cursor.execute(s)
            k=cursor.fetchall()
#             print(k)
            if(k==[]):
                num=re.findall(r"[-+]?(?:\d*\.*\d+)", name)
                strike=num[0]
                typecepe=name[-2:]
                sql3 ='''INSERT INTO "'''+schema+'''"."'''+infotable+'''"(Ticker,Strike,Type)VALUES (%s,%s,%s)'''
                record_to_insert = (name,strike,typecepe)
                cursor.execute(sql3, record_to_insert)
                conn.commit()
            sql4='''Select count(*) from "'''+schema+'''"."'''+name+'''" where date=\''''+datestr+'''\';'''
            cursor.execute(sql4)
            r=cursor.fetchall()
#             print(r[0][0])
            if(r[0][0]!=noofrows):
                print("numberofrows not matching")
                break
    conn.close()
    
def nifty_weekly():
    conn = psycopg2.connect(database="Niftydb",
                            user='postgres', password='swancap123',
                            host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com', port='5432'
    )
    conn.autocommit = True
    cursor = conn.cursor()
    print("APPENDING NIFTY WEEKLY CONTRACTS")
    for i in range(1,15):
        if(i==1):
            numberstring="I"
            schema="NiftyWeeklyI"
            infotable="NIFTYWEEKLY-IInfo"
        if(i==2):
            numberstring="II"
            schema="NiftyWeeklyII"
            infotable="NIFTYWEEKLY-IIInfo"
        if(i==3):
            numberstring="III"
            schema="NiftyWeeklyIII"
            infotable="NIFTYWEEKLY-IIIInfo"
        if(i==4):
            numberstring="IV"
            schema="NiftyWeeklyIV"
            infotable="NIFTYWEEKLY-IVInfo"
        if(i==5):
            numberstring="V"
            schema="NiftyWeeklyV"
            infotable="NIFTYWEEKLY-VInfo"
        if(i==6):
            numberstring="VI"
            schema="NiftyWeeklyVI"
            infotable="NIFTYWEEKLY-VIInfo"
        if(i==7):
            numberstring="VII"
            schema="NiftyWeeklyVII"
            infotable="NIFTYWEEKLY-VIIInfo"
        if(i==8):
            numberstring="VIII"
            schema="NiftyWeeklyVIII"
            infotable="NIFTYWEEKLY-VIIIInfo"
        if(i==9):
            numberstring="IX"
            schema="NiftyWeeklyIX"
            infotable="NIFTYWEEKLY-IXInfo"
        if(i==10):
            numberstring="X"
            schema="NiftyWeeklyX"
            infotable="NIFTYWEEKLY-XInfo"
        if(i==11):
            numberstring="XI"
            schema="NiftyWeeklyXI"
            infotable="NIFTYWEEKLY-XIInfo"
        if(i==12):
            numberstring="XII"
            schema="NiftyWeeklyXII"
            infotable="NIFTYWEEKLY-XIIInfo"
        if(i==13):
            numberstring="XIII"
            schema="NiftyWeeklyXIII"
            infotable="NIFTYWEEKLY-XIIIInfo"
        if(i==14):
            numberstring="XIV"
            schema="NiftyWeeklyXIV"
            infotable="NIFTYWEEKLY-XIVInfo"
        if os.path.exists(r"C:\Users\admin\Desktop\Pyspark\Nifty\Weekly\NIFTY-"+numberstring+".csv"):
            bndf=pd.read_csv(r"C:\Users\admin\Desktop\Pyspark\Nifty\Weekly\NIFTY-"+numberstring+".csv")
            df=bndf.groupby(['Ticker'])
            for name,group in df:
                print(name)
                sql2='''CREATE TABLE IF NOT EXISTS "'''+schema+'''"."'''+name+'''"(Ticker varchar(50) NOT NULL,Date Date NOT NULL,Time time NOT NULL,Open float NOT NULL,High float NOT NULL,Low float NOT NULL,Close float NOT NULL,Volume float NOT NULL,"Open Int" float NOT NULL);'''
                cursor.execute(sql2)
                conn.commit()
                noofrows=group.shape[0]
#                 print(noofrows)
                buffer = StringIO()
                group.to_csv(buffer, index = False)
                buffer.seek(0)
                sql = "COPY %s FROM STDIN WITH CSV HEADER DELIMITER AS ','"
                table='"'+schema+'"."'+name+'"'
                with conn.cursor() as cur:
                    #cur.execute("truncate " + table + ";")
                    cur.copy_expert(sql=sql % table, file=buffer)
                    conn.commit()
                cursor = conn.cursor()
                s='''Select 1 from "'''+schema+'''"."'''+infotable+'''" where ticker=\''''+name+'''\';'''
                cursor.execute(s)
                k=cursor.fetchall()
#                 print(k)
                if(k==[]):
                    num=re.findall(r"[-+]?(?:\d*\.*\d+)", name)
                    strike=num[0]
                    typecepe=name[-2:]
                    sql3 ='''INSERT INTO "'''+schema+'''"."'''+infotable+'''"(Ticker,Strike,Type)VALUES (%s,%s,%s)'''
                    record_to_insert = (name,strike,typecepe)
                    cursor.execute(sql3, record_to_insert)
                    conn.commit()
                sql4='''Select count(*) from "'''+schema+'''"."'''+name+'''" where date=\''''+datestr+'''\';'''
                cursor.execute(sql4)
                r=cursor.fetchall()
#                 print(r[0][0])
                if(r[0][0]!=noofrows):
                    print("numberofrows not matching")
                    break
    conn.close()
    
def nifty_quarterly():
    conn = psycopg2.connect(database="Niftydb",
                            user='postgres', password='swancap123',
                            host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com', port='5432'
    )
    conn.autocommit = True
    cursor = conn.cursor()
    print("APPENDING NIFTY QUARTERLY CONTRACTS")
    for i in range(1,5):
        if(i==1):
            numberstring="I"
            schema="NiftyQuarterlyI"
            infotable="NIFTYQUARTERLY-Iinfo"
        if(i==2):
            numberstring="II"
            schema="NiftyQuarterlyII"
            infotable="NIFTYQUARTERLY-IIinfo"
        if(i==3):
            numberstring="III"
            schema="NiftyQuarterlyIII"
            infotable="NIFTYQUARTERLY-IIIinfo"
        if(i==4):
            numberstring="IV"
            schema="NiftyQuarterlyIV"
            infotable="NIFTYQUARTERLY-IVinfo"
        if os.path.exists(r"C:\Users\admin\Desktop\Pyspark\Nifty\Quarterly\NIFTY-"+numberstring+".csv"):
            bndf=pd.read_csv(r"C:\\Users\\admin\\Desktop\\Pyspark\Nifty\Quarterly\\NIFTY-"+numberstring+".csv")
            df=bndf.groupby(['Ticker'])
            for name,group in df:
                print(name)
                sql2='''CREATE TABLE IF NOT EXISTS "'''+schema+'''"."'''+name+'''"(Ticker varchar(50) NOT NULL,Date Date NOT NULL,Time time NOT NULL,Open float NOT NULL,High float NOT NULL,Low float NOT NULL,Close float NOT NULL,Volume float NOT NULL,"Open Int" float NOT NULL);'''
                cursor.execute(sql2)
                conn.commit()
                noofrows=group.shape[0]
#                 print(noofrows)
                buffer = StringIO()
                group.to_csv(buffer, index = False)
                buffer.seek(0)
                sql = "COPY %s FROM STDIN WITH CSV HEADER DELIMITER AS ','"
                table='"'+schema+'"."'+name+'"'
                with conn.cursor() as cur:
                    #cur.execute("truncate " + table + ";")
                    cur.copy_expert(sql=sql % table, file=buffer)
                    conn.commit()
                cursor = conn.cursor()
                s='''Select 1 from "'''+schema+'''"."'''+infotable+'''" where ticker=\''''+name+'''\';'''
                cursor.execute(s)
                k=cursor.fetchall()
#                 print(k)
                if(k==[]):
                    num=re.findall(r"[-+]?(?:\d*\.*\d+)", name)
                    strike=num[0]
                    typecepe=name[-2:]
                    sql3 ='''INSERT INTO "'''+schema+'''"."'''+infotable+'''"(Ticker,Strike,Type)VALUES (%s,%s,%s)'''
                    record_to_insert = (name,strike,typecepe)
                    cursor.execute(sql3, record_to_insert)
                    conn.commit()
                sql4='''Select count(*) from "'''+schema+'''"."'''+name+'''" where date=\''''+datestr+'''\';'''
                cursor.execute(sql4)
                r=cursor.fetchall()
#                 print(r[0][0])
                if(r[0][0]!=noofrows):
                    print("numberofrows not matching")
                    break
    conn.close()
    
def nifty_halfyearly():
    conn = psycopg2.connect(database="Niftydb",
                            user='postgres', password='swancap123',
                            host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com', port='5432'
    )
    conn.autocommit = True
    cursor = conn.cursor()
    print("APPENDING NIFTY HALFYEARLY CONTRACTS")
    for i in range(1,11):
        if(i==1):
            numberstring="I"
            schema="NiftyHalfYearlyI"
            infotable="NIFTYHALFYEARLY-Iinfo"
        if(i==2):
            numberstring="II"
            schema="NiftyHalfYearlyII"
            infotable="NIFTYHALFYEARLY-IIinfo"
        if(i==3):
            numberstring="III"
            schema="NiftyHalfYearlyIII"
            infotable="NIFTYHALFYEARLY-IIIinfo"
        if(i==4):
            numberstring="IV"
            schema="NiftyHalfYearlyIV"
            infotable="NIFTYHALFYEARLY-IVinfo"
        if(i==5):
            numberstring="V"
            schema="NiftyHalfYearlyV"
            infotable="NIFTYHALFYEARLY-Vinfo"
        if(i==6):
            numberstring="VI"
            schema="NiftyHalfYearlyVI"
            infotable="NIFTYHALFYEARLY-VIinfo"
        if(i==7):
            numberstring="VII"
            schema="NiftyHalfYearlyVII"
            infotable="NIFTYHALFYEARLY-VIIinfo"
        if(i==8):
            numberstring="VIII"
            schema="NiftyHalfYearlyVIII"
            infotable="NIFTYHALFYEARLY-VIIIinfo"
        if(i==9):
            numberstring="IX"
            schema="NiftyHalfYearlyIX"
            infotable="NIFTYHALFYEARLY-IXinfo"
        if(i==10):
            numberstring="X"
            schema="NiftyHalfYearlyX"
            infotable="NIFTYHALFYEARLY-Xinfo"
        if os.path.exists(r"C:\Users\admin\Desktop\Pyspark\Nifty\Half_Yearly\NIFTY-"+numberstring+".csv"):
            bndf=pd.read_csv(r"C:\\Users\\admin\\Desktop\\Pyspark\Nifty\Half_Yearly\\NIFTY-"+numberstring+".csv")
            df=bndf.groupby(['ticker'])
            for name,group in df:
                print(name)
                sql2='''CREATE TABLE IF NOT EXISTS "'''+schema+'''"."'''+name+'''"(Ticker varchar(50) NOT NULL,Date Date NOT NULL,Time time NOT NULL,Open float NOT NULL,High float NOT NULL,Low float NOT NULL,Close float NOT NULL,Volume float NOT NULL,"Open Int" float NOT NULL);'''
                cursor.execute(sql2)
                conn.commit()
                noofrows=group.shape[0]
#                 print(noofrows)
                buffer = StringIO()
                group.to_csv(buffer, index = False)
                buffer.seek(0)
                sql = "COPY %s FROM STDIN WITH CSV HEADER DELIMITER AS ','"
                table='"'+schema+'"."'+name+'"'
                with conn.cursor() as cur:
                    #cur.execute("truncate " + table + ";")
                    cur.copy_expert(sql=sql % table, file=buffer)
                    conn.commit()
                cursor = conn.cursor()
                s='''Select 1 from "'''+schema+'''"."'''+infotable+'''" where ticker=\''''+name+'''\';'''
                cursor.execute(s)
                k=cursor.fetchall()
#                 print(k)
                if(k==[]):
                    num=re.findall(r"[-+]?(?:\d*\.*\d+)", name)
                    strike=num[0]
                    typecepe=name[-2:]
                    sql3 ='''INSERT INTO "'''+schema+'''"."'''+infotable+'''"(Ticker,Strike,Type)VALUES (%s,%s,%s)'''
                    record_to_insert = (name,strike,typecepe)
                    cursor.execute(sql3, record_to_insert)
                    conn.commit()
                sql4='''Select count(*) from "'''+schema+'''"."'''+name+'''" where date=\''''+datestr+'''\';'''
                cursor.execute(sql4)
                r=cursor.fetchall()
#                 print(r[0][0])
                if(r[0][0]!=noofrows):
                    print("numberofrows not matching")
                    break
    conn.close()
    
def nifty_yearly():
    conn = psycopg2.connect(database="Niftydb",
                            user='postgres', password='swancap123',
                            host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com', port='5432'
    )
    conn.autocommit = True
    cursor = conn.cursor()
    print("APPENDING NIFTY YEARLY CONTRACTS")
    for i in range(1,6):
        if(i==1):
            numberstring="I"
            schema="NiftyYearlyI"
            infotable="NIFTYYEARLY-Iinfo"
        if(i==2):
            numberstring="II"
            schema="NiftyYearlyII"
            infotable="NIFTYYEARLY-IIinfo"
        if(i==3):
            numberstring="III"
            schema="NiftyYearlyIII"
            infotable="NIFTYYEARLY-IIIinfo"
        if(i==4):
            numberstring="IV"
            schema="NiftyYearlyIV"
            infotable="NIFTYYEARLY-IVinfo"
        if(i==5):
            numberstring="V"
            schema="NiftyYearlyV"
            infotable="NIFTYYEARLY-Vinfo"
        if os.path.exists(r"C:\Users\admin\Desktop\Pyspark\Nifty\Yearly\NIFTY-"+numberstring+".csv"):
            bndf=pd.read_csv(r"C:\\Users\\admin\\Desktop\\Pyspark\Nifty\Yearly\\NIFTY-"+numberstring+".csv")
            df=bndf.groupby(['ticker'])
            for name,group in df:
                print(name)
                sql2='''CREATE TABLE IF NOT EXISTS "'''+schema+'''"."'''+name+'''"(Ticker varchar(50) NOT NULL,Date Date NOT NULL,Time time NOT NULL,Open float NOT NULL,High float NOT NULL,Low float NOT NULL,Close float NOT NULL,Volume float NOT NULL,"Open Int" float NOT NULL);'''
                cursor.execute(sql2)
                conn.commit()
                noofrows=group.shape[0]
#                 print(noofrows)
                buffer = StringIO()
                group.to_csv(buffer, index = False)
                buffer.seek(0)
                sql = "COPY %s FROM STDIN WITH CSV HEADER DELIMITER AS ','"
                table='"'+schema+'"."'+name+'"'
                with conn.cursor() as cur:
                    #cur.execute("truncate " + table + ";")
                    cur.copy_expert(sql=sql % table, file=buffer)
                    conn.commit()
                cursor = conn.cursor()
                s='''Select 1 from "'''+schema+'''"."'''+infotable+'''" where ticker=\''''+name+'''\';'''
                cursor.execute(s)
                k=cursor.fetchall()
#                 print(k)
                if(k==[]):
                    num=re.findall(r"[-+]?(?:\d*\.*\d+)", name)
                    strike=num[0]
                    typecepe=name[-2:]
                    sql3 ='''INSERT INTO "'''+schema+'''"."'''+infotable+'''"(Ticker,Strike,Type)VALUES (%s,%s,%s)'''
                    record_to_insert = (name,strike,typecepe)
                    cursor.execute(sql3, record_to_insert)
                    conn.commit()
                sql4='''Select count(*) from "'''+schema+'''"."'''+name+'''" where date=\''''+datestr+'''\';'''
                cursor.execute(sql4)
                r=cursor.fetchall()
#                 print(r[0][0])
                if(r[0][0]!=noofrows):
                    print("numberofrows not matching")
                    break
    conn.close()
    
def finnifty_monthly():
    conn = psycopg2.connect(database="FinNiftydb",
    						user='postgres', password='swancap123',
    						host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com', port='5432'
    )
    conn.autocommit = True
    cursor = conn.cursor()
    folpath = r"C:\Users\admin\Desktop\Pyspark\FinNifty\Monthly\\"
    sym='FINNIFTY-'
    print("APPENDING FINNIFTY MONTHLY CONTRACTS")
    for i in range(1,4):
        if(i==1):
            numberstring="I"
            schema="FinNiftyMonthlyI"
            infotable="FINNIFTYMONTHLY-Iinfo"
        if(i==2):
            numberstring="II"
            schema="FinNiftyMonthlyII"
            infotable="FINNIFTYMONTHLY-IIinfo"
        if(i==3):
            numberstring="III"
            schema="FinNiftyMonthlyIII"
            infotable="FINNIFTYMONTHLY-IIIinfo"
    
        if os.path.exists(folpath+sym+numberstring+".csv"):    
            bndf=pd.read_csv(folpath+sym+numberstring+".csv")
            df=bndf.groupby(['ticker'])
            for name,group in df:
                print(name)
                sql2='''CREATE TABLE IF NOT EXISTS "'''+schema+'''"."'''+name+'''"(Ticker varchar(50) NOT NULL,Date Date NOT NULL,Time time NOT NULL,Open float NOT NULL,High float NOT NULL,Low float NOT NULL,Close float NOT NULL,Volume float NOT NULL,"Open Int" float NOT NULL);'''
                cursor.execute(sql2)
                conn.commit()
                noofrows=group.shape[0]
#                 print(noofrows)
                buffer = StringIO()
                group.to_csv(buffer, index = False)
                buffer.seek(0)
                sql = "COPY %s FROM STDIN WITH CSV HEADER DELIMITER AS ','"
                table='"'+schema+'"."'+name+'"'
                with conn.cursor() as cur:
                    #cur.execute("truncate " + table + ";")
                    cur.copy_expert(sql=sql % table, file=buffer)
                    conn.commit()
                cursor = conn.cursor()
                s='''Select 1 from "'''+schema+'''"."'''+infotable+'''" where ticker=\''''+name+'''\';'''
                cursor.execute(s)
                k=cursor.fetchall()
#                 print(k)
                if(k==[]):
                    num=re.findall(r"[-+]?(?:\d*\.*\d+)", name)
                    strike=num[0]
                    typecepe=name[-2:]
                    sql3 ='''INSERT INTO "'''+schema+'''"."'''+infotable+'''"(Ticker,Strike,Type)VALUES (%s,%s,%s)'''
                    record_to_insert = (name,strike,typecepe)
                    cursor.execute(sql3, record_to_insert)
                    conn.commit()
                sql4='''Select count(*) from "'''+schema+'''"."'''+name+'''" where date=\''''+datestr+'''\';'''
                cursor.execute(sql4)
                r=cursor.fetchall()
#                 print(r[0][0])
                if(r[0][0]!=noofrows):
                    print("numberofrows not matching")
                    break
    conn.close()
        
def finnifty_weekly():
    conn = psycopg2.connect(database="FinNiftydb",
    						user='postgres', password='swancap123',
    						host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com', port='5432'
    )
    conn.autocommit = True
    cursor = conn.cursor()
    folpath = r"C:\Users\admin\Desktop\Pyspark\FinNifty\Weekly\\"
    sym='FINNIFTY-'
    print("APPENDING FINNIFTY WEEKLY CONTRACTS")
    for i in range(1,15):
        if(i==1):
            numberstring="I"
            schema="FinNiftyWeeklyI"
            infotable="FINNIFTYWEEKLY-Iinfo"
        if(i==2):
            numberstring="II"
            schema="FinNiftyWeeklyII"
            infotable="FINNIFTYWEEKLY-IIinfo"
        if(i==3):
            numberstring="III"
            schema="FinNiftyWeeklyIII"
            infotable="FINNIFTYWEEKLY-IIIinfo"
        if(i==4):
            numberstring="IV"
            schema="FinNiftyWeeklyIV"
            infotable="FINNIFTYWEEKLY-IVinfo"
        if(i==5):
            numberstring="V"
            schema="FinNiftyWeeklyV"
            infotable="FINNIFTYWEEKLY-Vinfo"
        if(i==6):
            numberstring="VI"
            schema="FinNiftyWeeklyVI"
            infotable="FINNIFTYWEEKLY-VIinfo"
        if(i==7):
            numberstring="VII"
            schema="FinNiftyWeeklyVII"
            infotable="FINNIFTYWEEKLY-VIIinfo"
        if(i==8):
            numberstring="VIII"
            schema="FinNiftyWeeklyVIII"
            infotable="FINNIFTYWEEKLY-VIIIinfo"
        if(i==9):
            numberstring="IX"
            schema="FinNiftyWeeklyIX"
            infotable="FINNIFTYWEEKLY-IXinfo"
        if(i==10):
            numberstring="X"
            schema="FinNiftyWeeklyX"
            infotable="FINNIFTYWEEKLY-Xinfo"
        if(i==11):
            numberstring="XI"
            schema="FinNiftyWeeklyXI"
            infotable="FINNIFTYWEEKLY-XIinfo"
        if(i==12):
            numberstring="XII"
            schema="FinNiftyWeeklyXII"
            infotable="FINNIFTYWEEKLY-XIIinfo"
        if(i==13):
            numberstring="XIII"
            schema="FinNiftyWeeklyXIII"
            infotable="FINNIFTYWEEKLY-XIIIinfo"
        if(i==14):
            numberstring="XIV"
            schema="FinNiftyWeeklyXIV"
            infotable="FINNIFTYWEEKLY-XIVinfo"

        if os.path.exists(folpath+sym+numberstring+".csv"):    
            bndf=pd.read_csv(folpath+sym+numberstring+".csv")
            df=bndf.groupby(['ticker'])
            for name,group in df:
                print(name)
                sql2='''CREATE TABLE IF NOT EXISTS "'''+schema+'''"."'''+name+'''"(Ticker varchar(50) NOT NULL,Date Date NOT NULL,Time time NOT NULL,Open float NOT NULL,High float NOT NULL,Low float NOT NULL,Close float NOT NULL,Volume float NOT NULL,"Open Int" float NOT NULL);'''
                cursor.execute(sql2)
                conn.commit()
                noofrows=group.shape[0]
#                 print(noofrows)
                buffer = StringIO()
                group.to_csv(buffer, index = False)
                buffer.seek(0)
                sql = "COPY %s FROM STDIN WITH CSV HEADER DELIMITER AS ','"
                table='"'+schema+'"."'+name+'"'
                with conn.cursor() as cur:
                    #cur.execute("truncate " + table + ";")
                    cur.copy_expert(sql=sql % table, file=buffer)
                    conn.commit()
                cursor = conn.cursor()
                s='''Select 1 from "'''+schema+'''"."'''+infotable+'''" where ticker=\''''+name+'''\';'''
                cursor.execute(s)
                k=cursor.fetchall()
#                 print(k)
                if(k==[]):
                    num=re.findall(r"[-+]?(?:\d*\.*\d+)", name)
                    strike=num[0]
                    typecepe=name[-2:]
                    sql3 ='''INSERT INTO "'''+schema+'''"."'''+infotable+'''"(Ticker,Strike,Type)VALUES (%s,%s,%s)'''
                    record_to_insert = (name,strike,typecepe)
                    cursor.execute(sql3, record_to_insert)
                    conn.commit()
                sql4='''Select count(*) from "'''+schema+'''"."'''+name+'''" where date=\''''+datestr+'''\';'''
                cursor.execute(sql4)
                r=cursor.fetchall()
#                 print(r[0][0])
                if(r[0][0]!=noofrows):
                    print("numberofrows not matching")
                    break
    conn.close()      
      

def option(num):
    if num==1:
        ## CONNECTION FOR BANKNIFTY
        def function_call():
            val = int(input("1 for MONTHLY\n2 for WEEKLY\n3 for QUARTERLY\n4 for HALF YEARLY\n5 for YEARLY\n6 to update ALL\n7 to EXIT\n"))
            if val==1:
                banknifty_monthly()
            elif val==2:
                banknifty_weekly()
            elif val==3:
                banknifty_quarterly()
            elif val==4:
                banknifty_halfyearly()
            elif val==5:
                banknifty_yearly()
            elif val==6:
                banknifty_monthly()
                banknifty_weekly()
                banknifty_quarterly()
                banknifty_halfyearly()
                banknifty_yearly()
            elif val==7:
                print("EXIT!!!")
    
            else:
                print("WRONG OPTION")
                function_call()
        function_call()
                
    elif num==2:
        ## CONNECTION FOR NIFTY
        def function_call():
            val = int(input("1 for MONTHLY\n2 for WEEKLY\n3 for QUARTERLY\n4 for HALF YEARLY\n5 for YEARLY\n6 to update ALL\n7 to EXIT\n"))
            if val==1:
                nifty_monthly()
            elif val==2:
                nifty_weekly()
            elif val==3:
                nifty_quarterly()
            elif val==4:
                nifty_halfyearly()
            elif val==5:
                nifty_yearly()
            elif val==6:
                nifty_monthly()
                nifty_weekly()
                nifty_quarterly()
                nifty_halfyearly()
                nifty_yearly()
            elif val==7:
                print("EXIT!!!")
    
            else:
                print("WRONG OPTION")
                function_call()
        function_call()
    
    elif num==3:
        ## CONNECTION FOR FINNIFTY
        def function_call():
            val = int(input("1 for MONTHLY\n2 for WEEKLY\n3 to update BOTH\n4 to EXIT\n"))
            if val==1:
                finnifty_monthly()
                    
            elif val==2:
                finnifty_weekly()
        
            elif val==3:
                finnifty_monthly()
                finnifty_weekly()
            elif val==4:
                print("EXIT!!!")

            else:
                print("WRONG OPTION")
                function_call()
        function_call()
        
    elif num==4:
        banknifty_monthly()
        banknifty_weekly()
        banknifty_quarterly()
        banknifty_halfyearly()
        banknifty_yearly()
        nifty_monthly()
        nifty_weekly()
        nifty_quarterly()
        nifty_halfyearly()
        nifty_yearly()
        finnifty_monthly()
        finnifty_weekly()
    
    elif num==5:
        print("BREAKING")
        
    else:
        print("Wrong option.\nPlease enter correct option")
        num = int(input("1 for BANKNIFTY\n2 for NIFTY\n3 for FINNIFTY\n4 to UPDATE ALL\n5 to EXIT\n"))
        return option(num)
    
option(choice)

et=time.time()

elapsed_time=et-st;
print("elapsed_time:",elapsed_time)


Enter a date in YYYY-MM-DD format 2023-05-09
2023-05-09
1 for BANKNIFTY
2 for NIFTY
3 for FINNIFTY
4 to UPDATE ALL
5 to EXIT
3
1 for MONTHLY
2 for WEEKLY
3 to update BOTH
4 to EXIT
3
APPENDING FINNIFTY MONTHLY CONTRACTS
FINNIFTYMONTHLY-I16500PE
FINNIFTYMONTHLY-I17000PE
FINNIFTYMONTHLY-I18000PE
FINNIFTYMONTHLY-I18350PE
FINNIFTYMONTHLY-I19000CE
FINNIFTYMONTHLY-I19000PE
FINNIFTYMONTHLY-I19100CE
FINNIFTYMONTHLY-I19150PE
FINNIFTYMONTHLY-I19200CE
FINNIFTYMONTHLY-I19200PE
FINNIFTYMONTHLY-I19250CE
FINNIFTYMONTHLY-I19300CE
FINNIFTYMONTHLY-I19300PE
FINNIFTYMONTHLY-I19350CE
FINNIFTYMONTHLY-I19400CE
FINNIFTYMONTHLY-I19400PE
FINNIFTYMONTHLY-I19500CE
FINNIFTYMONTHLY-I19500PE
FINNIFTYMONTHLY-I21500CE
APPENDING FINNIFTY WEEKLY CONTRACTS
FINNIFTYWEEKLY-I16200CE
FINNIFTYWEEKLY-I16200PE
FINNIFTYWEEKLY-I16300PE
FINNIFTYWEEKLY-I16400PE
FINNIFTYWEEKLY-I16500PE
FINNIFTYWEEKLY-I16600PE
FINNIFTYWEEKLY-I16700PE
FINNIFTYWEEKLY-I16800PE
FINNIFTYWEEKLY-I16900PE
FINNIFTYWEEKLY-I17000PE
FINNIFTYWEEKLY-I17100PE
FINNI